In [1]:
from __future__ import division
import networkx as nx
import matplotlib.pyplot as plt
import pylab
import scipy as sp
import numpy as np
import itertools
import copy
import math
import random

N =  50                  # Number of nodes
rw = 700                 # Wealth value of rich
pw = 300                # Wealth value of poor
we_arr = [rw, pw]       # Array storing both the wealth values
pr = 0.5                # Probability of being rich
prob_arr = [pr, 1-pr]  # Array storing probability of wealth allotment values
M = 50                  # Number of rounds
co = 50                 # Unitary Cost of co-operation
be = 100                # Unitary Benefit of co-operation
#re = 0.3                # Rewiring probability
S = 100                  # Number of iterations
pl = 0.3                # Initial probability of a link being formed between any two nodes


def inv_logit(y):
    inv_logit_out =(np.exp(y)/(1+np.exp(y)))
    return inv_logit_out

def gini_calc(x):       #IN: Wealth values across the population, OUT: Gini Coefficient
    sum2=0
    for i in range(len(x)):
        sum1=0
        for j in range(len(x)):
            sum1=sum1+np.abs(x[i]-x[j])
        sum2=sum2+sum1;
    
    gini=sum2/(2*len(x)**2*np.mean(x));
    return gini

node_arr = np.arange(N)
re_arr = [0,0.3,0.6,1.0]

rc_ar = np.zeros([len(re_arr),S,M])   # Array for storing no of cooperations by rich individuals
rd_ar = np.zeros([len(re_arr),S,M])   # Array for storing no of defections by rich individuals
pd_ar = np.zeros([len(re_arr),S,M])   # Array for storing no of defections by poor individuals
pc_ar = np.zeros([len(re_arr),S,M])   # Array for storing no of cooperations by poor individuals

flag = 0

for h1 in range(len(re_arr)):    
    re = re_arr[h1]
    for h in range(S):
        # Initializing the counts
        n_rc = 0
        n_rd = 0
        n_pd = 0
        n_pc = 0
        
        G = nx.erdos_renyi_graph(N, pl)
        A = nx.adjacency_matrix(G)
        E = G.number_of_edges()
        No_of_pairs = (N*(N-1))/2
        No_of_rewiring = round(No_of_pairs*re)
        Nre = int(No_of_rewiring)
        
        init_wealth_array = np.random.choice(we_arr,N,p=prob_arr) 
        GINI = gini_calc(init_wealth_array)        
        p0 = inv_logit((-1.017021)*GINI + (0.8130213)) 
            
        for k in range(M):                 # Continuing the game for M rounds
            n_rc = 0
            n_rd = 0
            n_pd = 0
            n_pc = 0
           
            if (k==0):                    # First round 
                
                decision_array = np.zeros(N)
                for i in range(N):         # For all nodes in the network
                    a = np.random.random()
                                   
                    if (a <= p0):
                        decision_array[i] = 1       # Node-i decides to be cooperative                    
                        init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                        for l in range(N):          # Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue

                    else:
                        decision_array[i] = 0          # Node-i decides to be non-cooperative
                        continue       
                        
                for i in range(N):
                    sum_neighbor_wealth = 0
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            sum_neighbor_wealth += init_wealth_array[j]

                    if (G.degree(i)>0):                            
                        diff_wealth = (sum_neighbor_wealth)/ (G.degree(i)) - (init_wealth_array[i])
                        if (decision_array[i]==1):
                            if (diff_wealth<0):
                                n_rc += 1
                            else:
                                n_pc += 1
                        elif (decision_array[i]==0): 
                            if (diff_wealth>0):
                                n_pd += 1
                            else:
                                n_rd += 1
                
                
            else:                     # Decision making in subsequent rounds
                new_decision_array = np.zeros(N)
                
                for i in range(N):         # For all nodes in the network                                    
                    cop = 0                     # Initializing the no of cooperator neighbours
                    sum_neighbor_wealth = 0
                    for j in range(N):
                        if (A[i,j] == 1):               # If a link exists between nodes i and j
                            sum_neighbor_wealth += init_wealth_array[j]
                            if (decision_array[j] == 1):        # If link j cooperated in last round
                                cop += 1
                    if (G.degree(i)>0):
                        diff_wealth = (sum_neighbor_wealth)/ (G.degree(i)) - (init_wealth_array[i])
                        pc = p0 + (1-p0)*(np.tanh(0.001*diff_wealth)) - 0.10 
                    else:
                        pc = p0                   
                
                
                    if (G.degree(i)>0):              # For postive number of nearest neighbours
                        cop_rat = float(cop/G.degree(i))                    
                        if (cop_rat > 0.5):       # If cooperation parameter is greater than 0.5 i.e. cooperative environment
                            
                            c = np.random.random()
                            if (c < pc):
                                new_decision_array[i] = 1       # Node-i decides to be cooperative                          
                            else:
                                new_decision_array[i] = 0

                        elif (cop_rat < 0.5):          # Cooperation parameter is lesser than 0.5 i.e. defective environment
                            
                            c = np.random.random()
                            if (c < pc):                        
                                new_decision_array[i] = 0
                            else:
                                new_decision_array[i] = 1
                            
                        elif (cop_rat == 0.5):       # Neutral environment
                              
                              c = np.random.random()
                              if (c<=pc):
                                  new_decision_array[i] = 1
                              else:
                                  new_decision_array[i] = 0
                                    
                        if (new_decision_array[i]==1):
                            if (diff_wealth<0):
                                n_rc += 1
                            else:
                                n_pc += 1
                        elif (new_decision_array[i]==0): 
                            if (diff_wealth>0):
                                n_pd += 1
                            else:
                                n_rd += 1
                                    
                    else:
                        new_decision_array[i] = 0
                        
                for i in range(N):
                    if (new_decision_array[i] == 1): 
                        init_wealth_array[i] = init_wealth_array[i] - co*(G.degree(i))   # Updatation of wealth of a cooperative node
                        for l in range(N):          	# Updatation of wealth of its neighbouring nodes
                            if (i==l):
                                continue
                            else:
                                if (A[i,l] == 1):
                                    init_wealth_array[l] = init_wealth_array[l] + be
                                else:
                                    continue
                    else:
                        continue       
                
                
                decision_array = copy.copy(new_decision_array)         
        
            # Rewiring step
                
            temp_array = []
                                                    
                                                                          
            while (len(temp_array) < Nre) :            
                i = random.choice(node_arr)
                j = random.choice(node_arr)
                    
                if (i==j):
                    continue
                else:
                
                    if (i,j) in temp_array:
                        continue
                    elif (j,i) in temp_array:
                        continue
                    else:
                        temp_array.append((i,j))
                        temp = [i,j]
                                                             
                        if (A[i,j] == 1):                              # If a link exists between the selected pair
                            b = random.choice(temp)
                            
                            if (b == i):                               # If node-i is given the choice to break or not
                                if (decision_array[j] == 0):
                                    c = np.random.random()
                                    if (c < 1.0):
                                        G.remove_edge(i,j)                 # Breakage of link for node-i disagreeing
                                  
                                    else:
                                        continue                        # Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 1.0):
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                            else:                                   # If node-j is given the choice to break or not	
                                if (decision_array[i] == 0):
                                    c = np.random.random()
                                    if (c < 1.0):                                    
                                        G.remove_edge(i,j)         # Breakage of link for node-j disagreeing
                                    else:
                                        continue			# Link is kept intact
                                else:
                                    c = np.random.random()
                                    if (c < 1.0):
                                        continue
                                    else:
                                        G.remove_edge(i,j)
                                                                
            
                        else:                 # If a link does not exist between the selected pair
                            if (decision_array[i] == 1 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 1.0):
                                    G.add_edge(i,j)
                                else:
                                    continue
                            
                            if (decision_array[i] == 0 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 1.0):
                                    continue
                                else:
                                    G.add_edge(i,j)
                        
                            if (decision_array[i] == 0 and decision_array[j] == 1):
                                c = np.random.random()
                                if (c < 1.0):
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
                            if (decision_array[i] == 1 and decision_array[j] == 0):
                                c = np.random.random()
                                if (c < 1.0):
                                    continue
                                else:
                                    G.add_edge(i,j)
                                
            A = nx.adjacency_matrix(G)     # Adjacency matrix of the network
            
                        
            rc_ar[h1,h,k] = n_rc
            rd_ar[h1,h,k] = n_rd
            pc_ar[h1,h,k] = n_pc
            pd_ar[h1,h,k] = n_pd
            
        flag += 1
        yo = (flag*100)/(len(re_arr)*S)
                        
        print('Job completion percentage: %f' %yo)

np.savez('S9a_strat,wealth_vs_rounds,p0=0.65,M=50', rc_ar=rc_ar,rd_ar=rd_ar,pc_ar=pc_ar,pd_ar=pd_ar)

Job completion percentage: 0.250000
Job completion percentage: 0.500000
Job completion percentage: 0.750000
Job completion percentage: 1.000000
Job completion percentage: 1.250000
Job completion percentage: 1.500000
Job completion percentage: 1.750000
Job completion percentage: 2.000000
Job completion percentage: 2.250000
Job completion percentage: 2.500000
Job completion percentage: 2.750000
Job completion percentage: 3.000000
Job completion percentage: 3.250000
Job completion percentage: 3.500000
Job completion percentage: 3.750000
Job completion percentage: 4.000000
Job completion percentage: 4.250000
Job completion percentage: 4.500000
Job completion percentage: 4.750000
Job completion percentage: 5.000000
Job completion percentage: 5.250000
Job completion percentage: 5.500000
Job completion percentage: 5.750000
Job completion percentage: 6.000000
Job completion percentage: 6.250000
Job completion percentage: 6.500000
Job completion percentage: 6.750000
Job completion percentage: 7

Job completion percentage: 56.000000
Job completion percentage: 56.250000
Job completion percentage: 56.500000
Job completion percentage: 56.750000
Job completion percentage: 57.000000
Job completion percentage: 57.250000
Job completion percentage: 57.500000
Job completion percentage: 57.750000
Job completion percentage: 58.000000
Job completion percentage: 58.250000
Job completion percentage: 58.500000
Job completion percentage: 58.750000
Job completion percentage: 59.000000
Job completion percentage: 59.250000
Job completion percentage: 59.500000
Job completion percentage: 59.750000
Job completion percentage: 60.000000
Job completion percentage: 60.250000
Job completion percentage: 60.500000
Job completion percentage: 60.750000
Job completion percentage: 61.000000
Job completion percentage: 61.250000
Job completion percentage: 61.500000
Job completion percentage: 61.750000
Job completion percentage: 62.000000
Job completion percentage: 62.250000
Job completion percentage: 62.500000
J